In [17]:
from copy import deepcopy
from itertools import combinations

# ------------------------------
# Unification Functions
# ------------------------------
def unify(x, y, subst):
    if subst is None:
        return None
    elif x == y:
        return subst
    elif isinstance(x, str) and x.islower():  # variable
        return unify_var(x, y, subst)
    elif isinstance(y, str) and y.islower():  # variable
        return unify_var(y, x, subst)
    elif isinstance(x, list) and isinstance(y, list) and len(x) == len(y):
        return unify(x[1:], y[1:], unify(x[0], y[0], subst))
    else:
        return None

def unify_var(var, x, subst):
    if var in subst:
        return unify(subst[var], x, subst)
    elif x in subst:
        return unify(var, subst[x], subst)
    else:
        subst_copy = deepcopy(subst)
        subst_copy[var] = x
        return subst_copy

def substitute(clause, subst):
    result = []
    for literal in clause:
        pred, args = literal[0], literal[1]
        new_args = [subst.get(arg, arg) for arg in args]
        result.append((pred, new_args))
    return result

# ------------------------------
# Resolution Logic
# ------------------------------
def resolve(ci, cj):
    """Resolve two clauses and return the resolvent if possible."""
    for di in ci:
        for dj in cj:
            if di[0] == f'¬{dj[0]}' or f'¬{di[0]}' == dj[0]:
                subst = {}
                args1, args2 = di[1], dj[1]
                if len(args1) == len(args2):
                    s = unify(args1, args2, subst)
                    if s is not None:
                        new_ci = [x for x in ci if x != di]
                        new_cj = [x for x in cj if x != dj]
                        new_clause = substitute(new_ci + new_cj, s)
                        return new_clause
    return None

def clause_to_str(clause):
    """Pretty print a clause."""
    return " ∨ ".join([f"{p}({', '.join(a)})" for p, a in clause]) if clause else "{}"

# ------------------------------
# Resolution with Tracing
# ------------------------------
def resolution(KB, query):
    clauses = deepcopy(KB)
    neg_query = [(f'¬{query[0]}', query[1])]
    clauses.append(neg_query)

    parents = {}
    step = 1

    print("\n=== Initial Clauses (Knowledge Base + ¬Query) ===")
    for i, c in enumerate(clauses):
        print(f"C{i+1}: {clause_to_str(c)}")

    print("\n=== Resolution Process ===")

    new = []
    while True:
        pairs = list(combinations(range(len(clauses)), 2))
        no_new = True

        for (i, j) in pairs:
            resolvent = resolve(clauses[i], clauses[j])
            if resolvent is not None:
                resolvent_str = clause_to_str(resolvent)
                print(f"\nStep {step}: Resolve C{i+1} and C{j+1}")
                print(f"     {clause_to_str(clauses[i])}")
                print(f"     {clause_to_str(clauses[j])}")
                print(f"  → Inferred: {resolvent_str}")
                step += 1

                if resolvent == []:
                    print("\n🎯 Empty clause derived → Contradiction found!")
                    parents["{}"] = (i+1, j+1)
                    print_resolution_tree("{}", parents, clauses)
                    return True

                if resolvent not in clauses:
                    clauses.append(resolvent)
                    new.append(resolvent)
                    parents[resolvent_str] = (i+1, j+1)
                    no_new = False

        if no_new:
            print("\n❌ No new clauses could be generated — Query cannot be proven.")
            print_resolution_tree("{}", parents, clauses)
            return False

# ------------------------------
# Pretty Tree Printer
# ------------------------------
def print_resolution_tree(root, parents, clauses, indent="", last=True, seen=None):
    """Recursively print resolution tree with indentation."""
    if seen is None:
        seen = set()

    branch = "└── " if last else "├── "
    print(indent + branch + root)

    if root not in parents:
        return

    left_id, right_id = parents[root]
    left_clause = clause_to_str(clauses[left_id - 1])
    right_clause = clause_to_str(clauses[right_id - 1])

    next_indent = indent + ("    " if last else "│   ")
    print_resolution_tree(left_clause, parents, clauses, next_indent, False, seen)
    print_resolution_tree(right_clause, parents, clauses, next_indent, True, seen)

# ------------------------------
# Knowledge Base (FOL Clauses)
# ------------------------------
KB = [
    [('¬food', ['x']), ('likes', ['John', 'x'])],   # ∀x food(x) → likes(John, x)
    [('food', ['Apple'])],
    [('food', ['Vegetables'])],
    [('¬eats', ['x', 'y']), ('killed', ['x']), ('food', ['y'])],  # eats(x,y) ∧ ¬killed(x) → food(y)
    [('eats', ['Anil', 'Peanuts'])],
    [('alive', ['Anil'])],
    [('¬eats', ['Anil', 'x']), ('eats', ['Harry', 'x'])],
    [('¬alive', ['x']), ('¬killed', ['x'])],
    [('killed', ['x']), ('alive', ['x'])]
]

query = ('likes', ['John', 'Peanuts'])

# ------------------------------
# Run Resolution
# ------------------------------
result = resolution(KB, query)
print("\n✅ John likes peanuts!" if result else "\n❌ Query cannot be proven.")


=== Initial Clauses (Knowledge Base + ¬Query) ===
C1: ¬food(x) ∨ likes(John, x)
C2: food(Apple)
C3: food(Vegetables)
C4: ¬eats(x, y) ∨ killed(x) ∨ food(y)
C5: eats(Anil, Peanuts)
C6: alive(Anil)
C7: ¬eats(Anil, x) ∨ eats(Harry, x)
C8: ¬alive(x) ∨ ¬killed(x)
C9: killed(x) ∨ alive(x)
C10: ¬likes(John, Peanuts)

=== Resolution Process ===

Step 1: Resolve C1 and C2
     ¬food(x) ∨ likes(John, x)
     food(Apple)
  → Inferred: likes(John, Apple)

Step 2: Resolve C1 and C3
     ¬food(x) ∨ likes(John, x)
     food(Vegetables)
  → Inferred: likes(John, Vegetables)

Step 3: Resolve C1 and C4
     ¬food(x) ∨ likes(John, x)
     ¬eats(x, y) ∨ killed(x) ∨ food(y)
  → Inferred: likes(John, y) ∨ ¬eats(y, y) ∨ killed(y)

Step 4: Resolve C1 and C10
     ¬food(x) ∨ likes(John, x)
     ¬likes(John, Peanuts)
  → Inferred: ¬food(Peanuts)

Step 5: Resolve C4 and C5
     ¬eats(x, y) ∨ killed(x) ∨ food(y)
     eats(Anil, Peanuts)
  → Inferred: killed(Anil) ∨ food(Peanuts)

Step 6: Resolve C4 and C7
     ¬e

# Task
Modify the python program for first order logic resolution to show the resolution tree.

## Modify `resolution` function

### Subtask:
Update the `resolution` function to store the history of resolution steps, including the parent clauses and the resulting resolvent.


**Reasoning**:
I need to modify the `resolution` function to store the history of resolution steps. I will initialize a list to store tuples of (parent_clause_index_1, parent_clause_index_2, resolvent).



In [10]:
def resolution(KB, query):
    """Resolution algorithm for FOL."""
    clauses = deepcopy(KB)
    # Negate query
    neg_query = [(f'¬{query[0]}', query[1])]
    clauses.append(neg_query)

    history = [] # Initialize history list

    new = []
    while True:
        n = len(clauses)
        for i in range(n):
            for j in range(i + 1, n):
                resolvent = resolve(clauses[i], clauses[j])
                if resolvent == []:
                    # Record the empty clause resolution in history
                    history.append((i, j, resolvent))
                    return True, history  # Empty clause found, return history
                if resolvent and resolvent not in clauses:
                    new.append(resolvent)
                    # Record the resolution step in history
                    history.append((i, j, resolvent))

        if set(map(str, new)).issubset(set(map(str, clauses))):
            return False, history # Return history even if no empty clause is found

        for c in new:
            if c not in clauses:
                clauses.append(c)

# ---------------------------------------------
# Knowledge Base (KB)
# ---------------------------------------------
KB = [
    [('¬food', ['x']), ('likes', ['John', 'x'])],   # ∀x food(x) → likes(John, x)
    [('food', ['Apple'])],
    [('food', ['Vegetables'])],
    [('¬eats', ['x', 'y']), ('killed', ['x']), ('food', ['y'])],  # eats(x,y) ∧ ¬killed(x) → food(y)
    [('eats', ['Anil', 'Peanuts'])],
    [('alive', ['Anil'])],
    [('¬eats', ['Anil', 'x']), ('eats', ['Harry', 'x'])],
    [('¬alive', ['x']), ('¬killed', ['x'])],
    [('killed', ['x']), ('alive', ['x'])]
]

# Query: likes(John, Peanuts)
query = ('likes', ['John', 'Peanuts'])

# ---------------------------------------------
# Run Resolution
# ---------------------------------------------
result, history = resolution(KB, query)
print("✅ John LIKE NUTS!🥜" if result else "❌ Query cannot be proven.")
print("Resolution History:")
for step in history:
    print(f"Parents: {step[0]}, {step[1]} -> Resolvent: {step[2]}")

✅ John LIKE NUTS!🥜
Resolution History:
Parents: 0, 1 -> Resolvent: [('likes', ['John', 'Apple'])]
Parents: 0, 2 -> Resolvent: [('likes', ['John', 'Vegetables'])]
Parents: 0, 3 -> Resolvent: [('likes', ['John', 'y']), ('¬eats', ['y', 'y']), ('killed', ['y'])]
Parents: 0, 9 -> Resolvent: [('¬food', ['Peanuts'])]
Parents: 3, 4 -> Resolvent: [('killed', ['Anil']), ('food', ['Peanuts'])]
Parents: 3, 6 -> Resolvent: [('killed', ['Harry']), ('food', ['Harry']), ('¬eats', ['Anil', 'Harry'])]
Parents: 3, 7 -> Resolvent: [('¬eats', ['x', 'y']), ('food', ['y']), ('¬alive', ['x'])]
Parents: 4, 6 -> Resolvent: [('eats', ['Harry', 'Peanuts'])]
Parents: 5, 7 -> Resolvent: [('¬killed', ['Anil'])]
Parents: 7, 8 -> Resolvent: [('¬killed', ['x']), ('killed', ['x'])]
Parents: 0, 14 -> Resolvent: [('likes', ['John', 'Peanuts']), ('killed', ['Anil'])]
Parents: 0, 15 -> Resolvent: [('likes', ['John', 'Harry']), ('killed', ['Harry']), ('¬eats', ['Anil', 'Harry'])]
Parents: 0, 16 -> Resolvent: [('likes', ['Joh

## Represent the tree

### Subtask:
Choose a suitable data structure to represent the resolution tree based on the stored history. This could be a dictionary, a list of tuples, or a dedicated tree structure.


**Reasoning**:
I will define a class to represent a node in the resolution tree and then create a list of these nodes to represent the tree structure. Each node will store its clause, its index in the list, and the indices of its parent nodes. I will then iterate through the `history` to populate this list of nodes.



In [11]:
class ResolutionNode:
    def __init__(self, index, clause, parent1_index=None, parent2_index=None):
        self.index = index
        self.clause = clause
        self.parent1_index = parent1_index
        self.parent2_index = parent2_index

# Initialize the tree structure with the initial clauses from KB and negated query
tree = []
initial_clauses = deepcopy(KB)
initial_clauses.append([(f'¬{query[0]}', query[1])])

for i, clause in enumerate(initial_clauses):
    tree.append(ResolutionNode(i, clause))

# Populate the tree structure using the history
# Keep track of clauses added to the tree to assign indices
clause_to_index = {str(clause): i for i, clause in enumerate(initial_clauses)}
current_index = len(initial_clauses)

for parent1_index, parent2_index, resolvent in history:
    # Add the resolvent as a new node if it's not already in the tree
    if str(resolvent) not in clause_to_index:
        tree.append(ResolutionNode(current_index, resolvent, parent1_index, parent2_index))
        clause_to_index[str(resolvent)] = current_index
        current_index += 1

# The tree structure is now represented by the 'tree' list of ResolutionNode objects.
# Each node has its clause and references to its parents (by index).

## Visualize the tree

### Subtask:
Implement a function to visualize the resolution tree. This might involve using a library like `graphviz` or simply printing a text-based representation.


**Reasoning**:
Implement the `visualize_resolution_tree` function to print a text-based representation of the resolution tree.



In [12]:
def visualize_resolution_tree(tree):
    """Prints a text-based representation of the resolution tree."""
    print("\nResolution Tree:")
    for node in tree:
        parent_info = ""
        if node.parent1_index is not None and node.parent2_index is not None:
            parent_info = f" (Parents: {node.parent1_index}, {node.parent2_index})"
        print(f"Node {node.index}: {node.clause}{parent_info}")

# Visualize the generated tree
visualize_resolution_tree(tree)


Resolution Tree:
Node 0: [('¬food', ['x']), ('likes', ['John', 'x'])]
Node 1: [('food', ['Apple'])]
Node 2: [('food', ['Vegetables'])]
Node 3: [('¬eats', ['x', 'y']), ('killed', ['x']), ('food', ['y'])]
Node 4: [('eats', ['Anil', 'Peanuts'])]
Node 5: [('alive', ['Anil'])]
Node 6: [('¬eats', ['Anil', 'x']), ('eats', ['Harry', 'x'])]
Node 7: [('¬alive', ['x']), ('¬killed', ['x'])]
Node 8: [('killed', ['x']), ('alive', ['x'])]
Node 9: [('¬likes', ['John', 'Peanuts'])]
Node 10: [('likes', ['John', 'Apple'])] (Parents: 0, 1)
Node 11: [('likes', ['John', 'Vegetables'])] (Parents: 0, 2)
Node 12: [('likes', ['John', 'y']), ('¬eats', ['y', 'y']), ('killed', ['y'])] (Parents: 0, 3)
Node 13: [('¬food', ['Peanuts'])] (Parents: 0, 9)
Node 14: [('killed', ['Anil']), ('food', ['Peanuts'])] (Parents: 3, 4)
Node 15: [('killed', ['Harry']), ('food', ['Harry']), ('¬eats', ['Anil', 'Harry'])] (Parents: 3, 6)
Node 16: [('¬eats', ['x', 'y']), ('food', ['y']), ('¬alive', ['x'])] (Parents: 3, 7)
Node 17: [('

## Integrate visualization

### Subtask:
Call the visualization function after the `resolution` function completes to display the tree.


**Reasoning**:
Call the visualization function after the resolution function completes to display the tree.



In [13]:
# The result and history are already obtained from the previous step where resolution function was called.
# tree structure is also already built in the previous step.

# Visualize the generated tree
visualize_resolution_tree(tree)


Resolution Tree:
Node 0: [('¬food', ['x']), ('likes', ['John', 'x'])]
Node 1: [('food', ['Apple'])]
Node 2: [('food', ['Vegetables'])]
Node 3: [('¬eats', ['x', 'y']), ('killed', ['x']), ('food', ['y'])]
Node 4: [('eats', ['Anil', 'Peanuts'])]
Node 5: [('alive', ['Anil'])]
Node 6: [('¬eats', ['Anil', 'x']), ('eats', ['Harry', 'x'])]
Node 7: [('¬alive', ['x']), ('¬killed', ['x'])]
Node 8: [('killed', ['x']), ('alive', ['x'])]
Node 9: [('¬likes', ['John', 'Peanuts'])]
Node 10: [('likes', ['John', 'Apple'])] (Parents: 0, 1)
Node 11: [('likes', ['John', 'Vegetables'])] (Parents: 0, 2)
Node 12: [('likes', ['John', 'y']), ('¬eats', ['y', 'y']), ('killed', ['y'])] (Parents: 0, 3)
Node 13: [('¬food', ['Peanuts'])] (Parents: 0, 9)
Node 14: [('killed', ['Anil']), ('food', ['Peanuts'])] (Parents: 3, 4)
Node 15: [('killed', ['Harry']), ('food', ['Harry']), ('¬eats', ['Anil', 'Harry'])] (Parents: 3, 6)
Node 16: [('¬eats', ['x', 'y']), ('food', ['y']), ('¬alive', ['x'])] (Parents: 3, 7)
Node 17: [('